<a href="https://colab.research.google.com/github/LetsCodePhysics/PhysicsOfMusic/blob/main/Music_Generation_via_Markov_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install ThinkDSP

In [ ]:
!git clone https://github.com/AllenDowney/ThinkDSP.git

Cloning into 'ThinkDSP'...
remote: Enumerating objects: 2505, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 2505 (delta 12), reused 23 (delta 10), pack-reused 2469
Receiving objects: 100% (2505/2505), 209.59 MiB | 27.93 MiB/s, done.
Resolving deltas: 100% (1365/1365), done.


# Set frequencies

The code cell below sets frequencies for a C-major scale.

In [ ]:
import sys
sys.path.insert(0, 'ThinkDSP/code/')
import thinkdsp
import matplotlib.pyplot as pyplot
import IPython
import numpy as np

c_freq  = 262.0
d_freq  = 294.8
e_freq  = 327.5
f_freq  = 349.3
g_freq  = 393.0
a_freq  = 436.7
b_freq  = 491.2
cp_freq = 524.0

notes = [c_freq,d_freq,e_freq,f_freq,g_freq,a_freq,b_freq,cp_freq]

# Define wave concatenation

The function below will append a series of notes.

In [ ]:
def wave_concatenate(waves):
  # waves = a list of wave objects to be concatenated
  out_wave = waves[0]
  next_start = out_wave.start + out_wave.ts[len(out_wave.ts)-1]
  for i_wave in range(1,len(waves)):
    waves[i_wave].ts += next_start
    out_wave.ts = np.concatenate((out_wave.ts,waves[i_wave].ts))
    out_wave.ys = np.concatenate((out_wave.ys,waves[i_wave].ys))
    next_start  = out_wave.start + out_wave.ts[len(out_wave.ts)-1]
  return out_wave

# Example: Create a scale.

As an example, the code cell below creates and plays a scale.

In [ ]:
# Create a list of waves based on the notes of the scale.
list_of_waves = []
amplitude = 0.5
duration = 0.5
for note_freq in notes:
  signal = thinkdsp.SinSignal(freq=note_freq, amp=amplitude)
  wave = signal.make_wave(duration=duration, start=0, framerate=44100)
  list_of_waves.append(wave)


# Concatenate the notes together.
composition = wave_concatenate(list_of_waves)

# Play wave
composition.play()
IPython.display.Audio('sound.wav')

Writing sound.wav


# Create Markov chain matrix

This matrix is defined such that `mcm[i,j]` = the probability that note i will be followed by note j.

In [ ]:
# mcm filled with 1.0s makes any note equally likely to be next.
mcm = np.array([
     # Probability to play a ...
     # c   d   e   f   g   a   b   cp  If you just played a...
    [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0], # c
    [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0], # d
    [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0], # e
    [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0], # f
    [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0], # g
    [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0], # a
    [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0], # b
    [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0], # cp
])

# mcm that preferences I-IV-V chords.
mcm = np.array([
     # Probability to play a ...
     # c   d   e   f   g   a   b   cp  If you just played a...
    [1.0,1.0,5.0,1.0,5.0,1.0,1.0,1.0], # c
    [1.0,1.0,1.0,1.0,5.0,1.0,5.0,1.0], # d
    [5.0,1.0,5.0,1.0,5.0,1.0,1.0,5.0], # e
    [5.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0], # f
    [1.0,5.0,1.0,1.0,1.0,1.0,5.0,1.0], # g
    [5.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0], # a
    [1.0,5.0,1.0,1.0,5.0,1.0,1.0,1.0], # b
    [1.0,1.0,5.0,1.0,5.0,1.0,1.0,1.0], # cp
])

# Normalize the probabilities such that total probability across each row is 1.
for i in range(len(mcm)):
  sum = 0
  for j in range(len(mcm)):
    sum += mcm[i,j]
  mcm[i,:] = mcm[i,:]/sum

print(mcm)

[[0.0625     0.0625     0.3125     0.0625     0.3125     0.0625
  0.0625     0.0625    ]
 [0.0625     0.0625     0.0625     0.0625     0.3125     0.0625
  0.3125     0.0625    ]
 [0.20833333 0.04166667 0.20833333 0.04166667 0.20833333 0.04166667
  0.04166667 0.20833333]
 [0.3125     0.0625     0.0625     0.0625     0.0625     0.3125
  0.0625     0.0625    ]
 [0.0625     0.3125     0.0625     0.0625     0.0625     0.0625
  0.3125     0.0625    ]
 [0.3125     0.0625     0.0625     0.3125     0.0625     0.0625
  0.0625     0.0625    ]
 [0.0625     0.3125     0.0625     0.0625     0.3125     0.0625
  0.0625     0.0625    ]
 [0.0625     0.0625     0.3125     0.0625     0.3125     0.0625
  0.0625     0.0625    ]]


# Now randomly select a chain of notes, turn them into waves, and concatenate them.

In [ ]:
note_num = 0
note_freq = notes[note_num]

number_of_notes = 24
amplitude = 0.5
duration = 0.5
list_of_waves = []
for n in range(number_of_notes):
  # Record the current note as a wave.
  signal = thinkdsp.SinSignal(freq=note_freq, amp=amplitude)
  wave = signal.make_wave(duration=duration, start=0, framerate=44100)
  list_of_waves.append(wave)
  # Select the next note.
  random_number = np.random.random() # Select a random number between 0 and 1.
  next = 0
  sum = 0 # Sum = the total probability up to note j.
  for j in range(len(mcm[note_num])):
    if sum < random_number < sum + mcm[note_num][j]:
      # Save j as the next note if random_number falls between the
      # total probability for j-1 and j.
      note_num = j
    sum = sum + mcm[note_num][j]
  note_freq = notes[note_num]

# How could you force it to end on the tonic?
note_freq = notes[0]
signal = thinkdsp.SinSignal(freq=note_freq, amp=amplitude)
wave = signal.make_wave(duration=duration, start=0, framerate=44100)
list_of_waves.append(wave)


# Concatenate the notes together.
composition = wave_concatenate(list_of_waves)

# Play wave
composition.play()
IPython.display.Audio('sound.wav')

Writing sound.wav
